In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the training data
train_df = pd.read_csv('/kaggle/input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')

train_df

In [ ]:
# Define the image size and batch size
image_size = (128, 128)
batch_size = 32

In [ ]:
# Split the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(
    train_df['image_name'], train_df['tags'], test_size=0.2, random_state=42)

train_files

In [ ]:
# Create an image data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    zoom_range=0.2,
    shear_range=0.2)

In [ ]:
# Load and preprocess the training images
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/kaggle/input/planet-understanding-the-amazon-from-space/train-tif-v2',
    x_col='image_name',
    y_col='tags',
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size,
    subset='training',
    interpolation='nearest')

In [ ]:
# Create the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(17, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator, epochs=10)

In [ ]:
# Evaluate the model on the validation set
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='train-jpg',
    x_col='image_name',
    y_col='labels',
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size)

loss, accuracy = model.evaluate(val_generator)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy)